# Installs

In [ ]:
%pip install torch==1.13.1+cu117 torchvision==0.14.1+cu117 torchtext==0.14.1 torchaudio==0.13.1 torchdata==0.5.1 --extra-index-url https://download.pytorch.org/whl/cu117 -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 GB 544.7 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.3/24.3 MB 61.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 90.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 68.0 MB/s eta 0:00:00



This may take a while

In [ ]:
!pip install wandb --quiet
!pip install python-Levenshtein -q
!git clone --recursive https://github.com/parlance/ctcdecode.git
!pip install wget -q
%cd ctcdecode
!pip install . -q
%cd ..

!pip install torchsummaryX -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 21.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.9/264.9 kB 21.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.4/177.4 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 10.9 MB/s eta 0:00:00
Cloning into 'ctcdecode'...
remote: Enumerating objects: 1102, done.
remote: Counting objects: 100% (39/39), done.
remote: Compressing objects: 100% (25/25), done.
remote: Total 1102 (delta 16), reused 32 (delta 14), pack-reused 1063
Receiving objects: 100% (1102/1102), 782.27 KiB | 6.41 MiB/s, done.
Resolving deltas: 100% (529/529), done.
Submodule 'third_party/ThreadPool' (https://github.com/progschj/ThreadPool.git) registered for path 'third_party/ThreadPool'
Submodule 'third_party/kenlm' (https://github.c

In [ ]:
'''
If torchsummaryX doesn't work, please run this cell. Alternatively, please refer to Piazza post @209 for more assistance:
'''

# !pip install torchsummaryx==1.1.0

"\nIf torchsummaryX doesn't work, please run this cell. Alternatively, please refer to Piazza post @209 for more assistance:\n"

# Imports

In [ ]:
import torch
import random
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
from torchsummaryX import summary
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence, pad_packed_sequence

import torchaudio.transforms as tat

from sklearn.metrics import accuracy_score
import gc

import zipfile
import pandas as pd
from tqdm import tqdm
import os
import datetime

# imports for decoding and distance calculation
import ctcdecode
import Levenshtein
from ctcdecode import CTCBeamDecoder

import warnings
warnings.filterwarnings('ignore')

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print("Device: ", device)

Device:  cuda


# Kaggle Setup

In [ ]:
!pip install --upgrade --force-reinstall --no-deps kaggle==1.5.8 -q
!mkdir /root/.kaggle

with open("/root/.kaggle/kaggle.json", "w+") as f:
    f.write('{"username":"alicele","key":""}') # TODO: Put your kaggle username & key here

!chmod 600 /root/.kaggle/kaggle.json

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.2/59.2 kB 846.4 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [ ]:

!kaggle competitions download -c hw3p2asr-s24

100% 3.74G/3.74G [00:37<00:00, 92.8MB/s]
100% 3.74G/3.74G [00:37<00:00, 107MB/s] 


In [ ]:
'''
This will take a couple minutes, but you should see at least the following:
11-785-s24-hw3p2  ctcdecode  hw3p2asr-s24.zip  sample_data
'''
!unzip -q hw3p2asr-s24.zip
!ls

11-785-s24-hw3p2  ctcdecode  hw3p2asr-s24.zip  sample_data


# Google Drive

In [ ]:
# from google.colab import drive
# drive.mount('/content/gdrive')

# Dataset and Dataloader

In [ ]:
# ARPABET PHONEME MAPPING
# DO NOT CHANGE

CMUdict_ARPAbet = {
    "" : " ",
    "[SIL]": "-", "NG": "G", "F" : "f", "M" : "m", "AE": "@",
    "R"    : "r", "UW": "u", "N" : "n", "IY": "i", "AW": "W",
    "V"    : "v", "UH": "U", "OW": "o", "AA": "a", "ER": "R",
    "HH"   : "h", "Z" : "z", "K" : "k", "CH": "C", "W" : "w",
    "EY"   : "e", "ZH": "Z", "T" : "t", "EH": "E", "Y" : "y",
    "AH"   : "A", "B" : "b", "P" : "p", "TH": "T", "DH": "D",
    "AO"   : "c", "G" : "g", "L" : "l", "JH": "j", "OY": "O",
    "SH"   : "S", "D" : "d", "AY": "Y", "S" : "s", "IH": "I",
    "[SOS]": "[SOS]", "[EOS]": "[EOS]"
}

CMUdict = list(CMUdict_ARPAbet.keys())
ARPAbet = list(CMUdict_ARPAbet.values())


PHONEMES = CMUdict[:-2]
LABELS = ARPAbet[:-2]

In [ ]:
# You might want to play around with the mapping as a sanity check here

### Train Data

In [ ]:
class AudioDataset(torch.utils.data.Dataset):

    # For this homework, we give you full flexibility to design your data set class.
    # Hint: The data from HW1 is very similar to this HW

    #TODO
    def __init__(self, root, phonemes=PHONEMES, partition="train-clean-100"):
        '''
        Initializes the dataset.

        INPUTS: What inputs do you need here?
        '''

        self.mfcc_dir = root + partition + "/mfcc/"
        self.transcript_dir = root + partition + "/transcript/"

        self.mfcc_files = sorted(os.listdir(self.mfcc_dir))
        self.transcript_files = sorted(os.listdir(self.transcript_dir))

        self.PHONEMES = PHONEMES

        self.mfccs, self.transcripts = [], []

        for i in range(len(self.mfcc_files)):

            mfcc        = np.load(self.mfcc_dir + self.mfcc_files[i])

            mfcc = (mfcc - np.mean(mfcc, axis=0)) / np.std(mfcc, axis=0)

            transcript  = np.load(self.transcript_dir + self.transcript_files[i])

            transcript = transcript[1:-1]

            transcript = np.array([self.PHONEMES.index(element) for element in transcript])
            self.mfccs.append(mfcc)
            self.transcripts.append(transcript)

        self.length = len(self.mfccs)

    def __len__(self):

        '''
        TODO: What do we return here?
        '''
        return self.length


    def __getitem__(self, ind):


        mfcc = self.mfccs[ind]
        transcript = self.transcripts[ind]

        return torch.FloatTensor(mfcc), torch.tensor(transcript)

    def collate_fn(self,batch):
        '''
        TODO:
        1.  Extract the features and labels from 'batch'
        2.  We will additionally need to pad both features and labels,
            look at pytorch's docs for pad_sequence
        3.  This is a good place to perform transforms, if you so wish.
            Performing them on batches will speed the process up a bit.
        4.  Return batch of features, labels, lenghts of features,
            and lengths of labels.
        '''
        batch_mfcc = [item[0] for item in batch]

        batch_transcript = [item[1] for item in batch]

        batch_mfcc_pad = pad_sequence(batch_mfcc, batch_first=True)
        lengths_mfcc = [len(seq) for seq in batch_mfcc]

        batch_transcript_pad = pad_sequence(batch_transcript, batch_first=True)
        lengths_transcript = [len(seq) for seq in batch_transcript]

        return batch_mfcc_pad, batch_transcript_pad, torch.tensor(lengths_mfcc), torch.tensor(lengths_transcript)



### Test Data

In [ ]:
# Test Dataloader
#TODO

class AudioDatasetTest(torch.utils.data.Dataset):

    # For this homework, we give you full flexibility to design your data set class.
    # Hint: The data from HW1 is very similar to this HW

    #TODO
    def __init__(self, root, partition="test-clean"):
        '''
        Initializes the dataset.

        INPUTS: What inputs do you need here?
        '''



        self.mfcc_dir = root + partition + "/mfcc/"

        self.mfcc_files = sorted(os.listdir(self.mfcc_dir))

        # self.PHONEMES = PHONEMES

        self.mfccs = []

        for i in range(len(self.mfcc_files)):

            mfcc        = np.load(self.mfcc_dir + self.mfcc_files[i])

            mfcc = (mfcc - np.mean(mfcc, axis=0)) / np.std(mfcc, axis=0)

            self.mfccs.append(mfcc)

        self.length = len(self.mfccs)

        '''
        You may decide to do this in __getitem__ if you wish.
        However, doing this here will make the __init__ function take the load of
        loading the data, and shift it away from training.
        '''


    def __len__(self):

        '''
        TODO: What do we return here?
        '''
        return self.length

    def __getitem__(self, ind):


        mfcc = self.mfccs[ind]

        return torch.FloatTensor(mfcc)


    def collate_fn(self,batch):
        '''
        TODO:
        1.  Extract the features and labels from 'batch'
        2.  We will additionally need to pad both features and labels,
            look at pytorch's docs for pad_sequence
        3.  This is a good place to perform transforms, if you so wish.
            Performing them on batches will speed the process up a bit.
        4.  Return batch of features, labels, lenghts of features,
            and lengths of labels.
        '''
        batch_mfcc = batch

        batch_mfcc_pad = pad_sequence(batch_mfcc, batch_first=True)
        lengths_mfcc = [len(seq) for seq in batch_mfcc]

        return batch_mfcc_pad, torch.tensor(lengths_mfcc)



### Config - Hyperparameters

In [ ]:
# root = '/content/11-785-s24-hw3p2/'

# Feel free to add more items here
config = {
    "beam_width" : 2,
    "lr"         : 2e-3,
    "epochs"     : 40,
    "batch_size" : 128,  # Increase if your device can handle it
    "weight_decay" : 1e-4
}

# You may pass this as a parameter to the dataset class above
# This will help modularize your implementation
transforms = [] # set of tranformations

### Data loaders

In [ ]:
# get me RAMMM!!!!
import gc
gc.collect()

0

In [ ]:
root_data_path  = "/content/11-785-s24-hw3p2/"
# Create objects for the dataset class
train_data = AudioDataset(root=root_data_path,
                          phonemes=PHONEMES,
                          partition="train-clean-100") #TODO
val_data = AudioDataset(root=root_data_path,
                        phonemes=PHONEMES,
                        partition="dev-clean") # TODO : You can either use the same class with some modifications or make a new one :)
test_data = AudioDatasetTest(root=root_data_path,
                             partition="test-clean") #TODO

# Do NOT forget to pass in the collate function as parameter while creating the dataloader
train_loader = torch.utils.data.DataLoader(
    dataset     = train_data,
    num_workers = 4,
    batch_size  = config['batch_size'],
    pin_memory  = True,
    shuffle     = True,
    collate_fn  = train_data.collate_fn
)

val_loader = torch.utils.data.DataLoader(
    dataset     = val_data,
    num_workers = 2,
    batch_size  = config['batch_size'],
    pin_memory  = True,
    shuffle     = False,
    collate_fn  = val_data.collate_fn
)

test_loader = torch.utils.data.DataLoader(
    dataset     = test_data,
    num_workers = 2,
    batch_size  = config['batch_size'],
    pin_memory  = True,
    shuffle     = False,
    collate_fn  = test_data.collate_fn
)


print("Batch size: ", config['batch_size'])
print("Train dataset samples = {}, batches = {}".format(train_data.__len__(), len(train_loader)))
print("Val dataset samples = {}, batches = {}".format(val_data.__len__(), len(val_loader)))
print("Test dataset samples = {}, batches = {}".format(test_data.__len__(), len(test_loader)))

Batch size:  128
Train dataset samples = 28539, batches = 223
Val dataset samples = 2703, batches = 22
Test dataset samples = 2620, batches = 21


In [ ]:
# sanity check
for data in train_loader:
    x, y, lx, ly = data
    print(x.shape, y.shape, lx.shape, ly.shape)
    break

torch.Size([128, 1706, 27]) torch.Size([128, 208]) torch.Size([128]) torch.Size([128])


In [ ]:
 # sanity check
for data in test_loader:
    x, lx = data
    print(x.shape, lx.shape)
    break

torch.Size([128, 2363, 27]) torch.Size([128])


# NETWORK

## Basic

This is a basic block for understanding, you can skip this and move to pBLSTM one

In [ ]:
# torch.cuda.empty_cache()
# OUT_SIZE = len(PHONEMES)

# class Network(nn.Module):

#     def __init__(self, input_size):

#         super(Network, self).__init__()

#         # Adding some sort of embedding layer or feature extractor might help performance.
#         # self.embedding = ?

#         # TODO : look up the documentation. You might need to pass some additional parameters.
#         self.lstm = nn.LSTM(input_size = input_size, hidden_size = 256, num_layers = 1)

#         self.classification = nn.Sequential(
#             nn.Linear(in_features=256, out_features=OUT_SIZE)
#             #TODO: Linear layer with in_features from the lstm module above and out_features = OUT_SIZE
#         )
#         self.logSoftmax = nn.LogSoftmax(dim=-1)

#     def forward(self, x, lx):
#         #TODO
#         # The forward function takes 2 parameter inputs here. Why?
#         # Refer to the handout for hints
#         # Pack the sequence for LSTM processing
#         x_packed = nn.utils.rnn.pack_padded_sequence(x, lx, batch_first=True, enforce_sorted=False)

#         # LSTM and classification
#         lstm_out, (h_n, c_n) = self.lstm(x_packed)
#         lstm_out, _ = nn.utils.rnn.pad_packed_sequence(lstm_out, batch_first=True)

#         # You might need to select the last relevant output depending on your use case
#         output = self.classification(lstm_out)

#         # Apply logSoftmax
#         output = self.logSoftmax(output)

#         return output

## Initialize Basic Network
(If trying out the basic Network)

In [ ]:
# torch.cuda.empty_cache()

# model = Network(input_size = 27).to(device)
# summary(model, x.to(device), lx) # x and lx come from the sanity check above :)

## ASR Network

### Pyramid Bi-LSTM (pBLSTM)

In [ ]:
# Utils for network
torch.cuda.empty_cache()

class PermuteBlock(torch.nn.Module):
    def forward(self, x):
        return x.transpose(1, 2)

In [ ]:
class pBLSTM(torch.nn.Module):

    '''
    Pyramidal BiLSTM
    Read the write up/paper and understand the concepts and then write your implementation here.

    At each step,
    1. Pad your input if it is packed (Unpack it)
    2. Reduce the input length dimension by concatenating feature dimension
        (Tip: Write down the shapes and understand)
        (i) How should  you deal with odd/even length input?
        (ii) How should you deal with input length array (x_lens) after truncating the input?
    3. Pack your input
    4. Pass it into LSTM layer

    To make our implementation modular, we pass 1 layer at a time.
    '''




    def __init__(self, input_size, hidden_size):
        super(pBLSTM, self).__init__()
        # Each pBLSTM layer reduces the time resolution by half, hence we double the input_size
        self.blstm = torch.nn.LSTM(input_size=input_size * 2, hidden_size=hidden_size, num_layers=3, bidirectional=True, batch_first=True)


    def forward(self, x_packed):
        x_padded, x_lengths = pad_packed_sequence(x_packed, batch_first=True)

        x, new_lengths = self.trunc_reshape(x_padded, x_lengths)

        x_packed = pack_padded_sequence(x, lengths=new_lengths, batch_first=True, enforce_sorted=False)

        lstm_output, _ = self.blstm(x_packed)

        return lstm_output

    def trunc_reshape(self, x, x_lens):


        if x.shape[1] % 2 != 0:
            x = x[:, :-1]

        x = x.reshape(x.shape[0], x.shape[1] // 2, x.shape[2] * 2)

        x_lens = x_lens // 2

        return x, x_lens


### Encoder

In [ ]:
class Encoder(torch.nn.Module):
    '''
    The Encoder takes utterances as inputs and returns latent feature representations
    '''
    def __init__(self, input_size, encoder_hidden_size):
        super(Encoder, self).__init__()

        self.embedding = torch.nn.Sequential(
            PermuteBlock(),
            nn.Dropout1d(p = 0.1),
            nn.Conv1d(in_channels = input_size,out_channels = input_size*4,kernel_size=3,stride = 1,padding=1),
            nn.BatchNorm1d(input_size*4),
            nn.GELU(),
            nn.Dropout1d(p = 0.1),
            nn.Conv1d(in_channels = input_size*4,out_channels = input_size*8,kernel_size=3,stride = 1,padding=1),
            nn.BatchNorm1d(input_size*8),
            nn.GELU(),
            nn.Dropout1d(p = 0.1),
            nn.Conv1d(in_channels = input_size*8,out_channels = input_size*2,kernel_size=3,stride = 1,padding=1),
            nn.BatchNorm1d(input_size*2),
            nn.GELU(),
            PermuteBlock()
        )

        self.pBLSTMs = torch.nn.Sequential(
            pBLSTM(input_size=2*input_size, hidden_size=encoder_hidden_size),
            pBLSTM(input_size=2*encoder_hidden_size, hidden_size=encoder_hidden_size)
        )



    def forward(self, x, x_lens):
        x = self.embedding(x)

        x_packed = pack_padded_sequence(x, x_lens, batch_first=True, enforce_sorted=False)

        x_packed = self.pBLSTMs(x_packed)

        encoder_outputs, encoder_lens = pad_packed_sequence(x_packed, batch_first=True)

        return encoder_outputs, encoder_lens



### Decoder

In [ ]:
class Decoder(torch.nn.Module):

    def __init__(self, embed_size, output_size= 41):
        super().__init__()

        self.mlp = torch.nn.Sequential(
          nn.Linear(embed_size,1024),
          PermuteBlock(),
          nn.BatchNorm1d(1024),
          PermuteBlock(),
          nn.Dropout1d(p = 0.2),
          nn.Linear(1024,output_size),
          PermuteBlock(),
          nn.BatchNorm1d(output_size),
          PermuteBlock(),
      )

        self.softmax = torch.nn.LogSoftmax(dim=2)


    def forward(self, encoder_out):
        out = self.mlp(encoder_out)
        out = self.softmax(out)
        return out

In [ ]:
class ASRModel(torch.nn.Module):

    def __init__(self, input_size, embed_size= 192, output_size= len(PHONEMES)):
        super().__init__()

        self.augmentations  = torch.nn.Sequential(
            #TODO Add Time Masking/ Frequency Masking
            #Hint: See how to use PermuteBlock() function defined above
            # torchaudio.transforms.TimeMasking(time_mask_param=100),# experiment without
            # torchaudio.transforms.FrequencyMasking(freq_mask_param=27)
        )
        self.encoder        = Encoder(input_size=input_size, encoder_hidden_size=embed_size)# TODO: Initialize Encoder
        self.decoder        = Decoder(embed_size=embed_size*2, output_size=output_size) # TODO: Initialize Decoder



    def forward(self, x, lengths_x):

        if self.training:
            x = self.augmentations(x)

        encoder_out, encoder_lens   = self.encoder(x, lengths_x)
        decoder_out                 = self.decoder(encoder_out)

        return decoder_out, encoder_lens

## Initialize ASR Network

In [ ]:
import torchaudio
model = ASRModel(
    input_size  = 27,
    embed_size  = 192,
    output_size = len(PHONEMES)
).to(device)
print(model)
summary(model, x.to(device), lx)

ASRModel(
  (augmentations): Sequential()
  (encoder): Encoder(
    (embedding): Sequential(
      (0): PermuteBlock()
      (1): Dropout1d(p=0.1, inplace=False)
      (2): Conv1d(27, 108, kernel_size=(3,), stride=(1,), padding=(1,))
      (3): BatchNorm1d(108, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (4): GELU(approximate='none')
      (5): Dropout1d(p=0.1, inplace=False)
      (6): Conv1d(108, 216, kernel_size=(3,), stride=(1,), padding=(1,))
      (7): BatchNorm1d(216, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (8): GELU(approximate='none')
      (9): Dropout1d(p=0.1, inplace=False)
      (10): Conv1d(216, 54, kernel_size=(3,), stride=(1,), padding=(1,))
      (11): BatchNorm1d(54, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (12): GELU(approximate='none')
      (13): PermuteBlock()
    )
    (pBLSTMs): Sequential(
      (0): pBLSTM(
        (blstm): LSTM(108, 192, num_layers=3, batch_first=True, bidire

,Kernel Shape,Output Shape,Params,Mult-Adds
Layer,,,,
0_augmentations,-,"[128, 2363, 27]",NaN,NaN
1_encoder.embedding.PermuteBlock_0,-,"[128, 27, 2363]",NaN,NaN
2_encoder.embedding.Dropout1d_1,-,"[128, 27, 2363]",NaN,NaN
3_encoder.embedding.Conv1d_2,"[27, 108, 3]","[128, 108, 2363]",8856.0,20671524.0
4_encoder.embedding.BatchNorm1d_3,[108],"[128, 108, 2363]",216.0,108.0
5_encoder.embedding.GELU_4,-,"[128, 108, 2363]",NaN,NaN
6_encoder.embedding.Dropout1d_5,-,"[128, 108, 2363]",NaN,NaN
7_encoder.embedding.Conv1d_6,"[108, 216, 3]","[128, 216, 2363]",70200.0,165372192.0
8_encoder.embedding.BatchNorm1d_7,[216],"[128, 216, 2363]",432.0,216.0


# Training Config
Initialize Loss Criterion, Optimizer, CTC Beam Decoder, Scheduler, Scaler (Mixed-Precision), etc.

In [ ]:
#TODO
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.optim import AdamW
from ctcdecode import CTCBeamDecoder


criterion = torch.nn.CTCLoss(blank=0, reduction='mean', zero_infinity=False) # Define CTC loss as the criterion. How would the losses be reduced?
# CTC Loss: https://pytorch.org/docs/stable/generated/torch.nn.CTCLoss.html
# Refer to the handout for hints

optimizer =  torch.optim.AdamW(model.parameters(), lr=config['lr'], weight_decay=config['weight_decay']) # What goes in here?

# Declare the decoder. Use the CTC Beam Decoder to decode phonemes
# CTC Beam Decoder Doc: https://github.com/parlance/ctcdecode
decoder = CTCBeamDecoder(labels=PHONEMES,
                         model_path=None,
                         alpha=0,
                         beta=0,
                         cutoff_top_n=40,
                         cutoff_prob=1.0,
                         beam_width=config["beam_width"],
                         num_processes=4,
                         blank_id=0,
                         log_probs_input=True)#TODO

scheduler = ReduceLROnPlateau(optimizer, mode='max', threshold=1e-3, threshold_mode='rel', patience=5, factor = 0.2)#TODO

# Mixed Precision, if you need it
scaler = torch.cuda.amp.GradScaler()

# Decode Prediction

In [ ]:
def decode_prediction(output, output_lens, decoder, PHONEME_MAP= LABELS):

    # TODO: look at docs for CTC.decoder and find out what is returned here. Check the shape of output and expected shape in decode.
    beam_results, _, _, out_seq_len = decoder.decode(output, seq_lens= output_lens) #lengths - list of lengths

    pred_strings                    = []

    for i in range(beam_results.shape[0]):
        #TODO: Create the prediction from the output of decoder.decode. Don't forget to map it using PHONEMES_MAP.
        pred = beam_results[i][0][:out_seq_len[i][0]]  # Take most probable result (1st beam)
        pred_str = ''.join([PHONEME_MAP[p] for p in pred])
        pred_strings.append(pred_str)
    return pred_strings

def calculate_levenshtein(output, label, output_lens, label_lens, decoder, PHONEME_MAP= LABELS): # y - sequence of integers

    dist            = 0
    batch_size      = label.shape[0]

    pred_strings    = decode_prediction(output, output_lens, decoder, PHONEME_MAP)

    for i in range(batch_size):
        # TODO: Get predicted string and label string for each element in the batch
        pred_string = pred_strings[i]#TODO
        label_string = ''.join([PHONEME_MAP[p] for p in label[i][:label_lens[i]]])#TODO
        dist += Levenshtein.distance(pred_string, label_string)

    dist /= batch_size # TODO: Uncomment this, but think about why we are doing this
    # raise NotImplemented
    return dist

# Test Implementation

In [ ]:
# # test code to check shapes

# model.eval()
# for i, data in enumerate(val_loader, 0):
#     x, y, lx, ly = data
#     x, y = x.to(device), y.to(device)
#     h, lh = model(x, lx)
#     print(f'hshape {h.shape}')
#     h = torch.permute(h, (1, 0, 2))
#     print(f'hshape yshape {h.shape}, {y.shape}')
#     loss = criterion(h, y, lh, ly)
#     print(loss)

#     print(calculate_levenshtein(h, y, lx, ly, decoder, LABELS))

#     break

# WandB

You will need to fetch your api key from wandb.ai

In [ ]:
import wandb
wandb.login(key="f1ddfb597f3aa8c29d178a8df8d836349a961d46")

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
run = wandb.init(
    name = "ablation 5", ## Wandb creates random run names if you skip this field
    reinit = True, ### Allows reinitalizing runs when you re-run this cell
    # run_id = ### Insert specific run id here if you want to resume a previous run
    # resume = "must" ### You need this to resume previous runs, but comment out reinit = True when using this
    project = "hw3p2-ablations", ### Project should be created in your wandb account
    config = config ### Wandb Config for your run
)

wandb: Currently logged in as: alicele2710 (alicele27). Use `wandb login --relogin` to force relogin


# Train Functions

In [ ]:
from tqdm import tqdm

def train_model(model, train_loader, criterion, optimizer):

    model.train()
    batch_bar = tqdm(total=len(train_loader), dynamic_ncols=True, leave=False, position=0, desc='Train')

    total_loss = 0

    for i, data in enumerate(train_loader):
        optimizer.zero_grad()

        x, y, lx, ly = data
        x, y = x.to(device), y.to(device)

        with torch.cuda.amp.autocast():
            h, lh = model(x, lx)
            h = torch.permute(h, (1, 0, 2))
            loss = criterion(h, y, lh, ly)

        total_loss += loss.item()

        batch_bar.set_postfix(
            loss="{:.04f}".format(float(total_loss / (i + 1))),
            lr="{:.06f}".format(float(optimizer.param_groups[0]['lr'])))

        batch_bar.update() # Update tqdm bar

        # Another couple things you need for FP16.
        scaler.scale(loss).backward() # This is a replacement for loss.backward()
        scaler.step(optimizer) # This is a replacement for optimizer.step()
        scaler.update() # This is something added just for FP16

        del x, y, lx, ly, h, lh, loss
        torch.cuda.empty_cache()

    batch_bar.close() # You need this to close the tqdm bar

    return total_loss / len(train_loader)


def validate_model(model, val_loader, decoder, phoneme_map= LABELS):

    model.eval()
    batch_bar = tqdm(total=len(val_loader), dynamic_ncols=True, position=0, leave=False, desc='Val')

    total_loss = 0
    vdist = 0

    for i, data in enumerate(val_loader):

        x, y, lx, ly = data
        x, y = x.to(device), y.to(device)

        with torch.inference_mode():
            h, lh = model(x, lx)
            h = torch.permute(h, (1, 0, 2))
            loss = criterion(h, y, lh, ly)

        total_loss += float(loss)
        vdist += calculate_levenshtein(torch.permute(h, (1, 0, 2)), y, lh, ly, decoder, phoneme_map)

        batch_bar.set_postfix(loss="{:.04f}".format(float(total_loss / (i + 1))), dist="{:.04f}".format(float(vdist / (i + 1))))

        batch_bar.update()

        del x, y, lx, ly, h, lh, loss
        torch.cuda.empty_cache()

    batch_bar.close()
    total_loss = total_loss/len(val_loader)
    val_dist = vdist/len(val_loader)
    return total_loss, val_dist

## Training Setup

In [ ]:
def save_model(model, optimizer, scheduler, metric, epoch, path):
    torch.save(
        {'model_state_dict'         : model.state_dict(),
         'optimizer_state_dict'     : optimizer.state_dict(),
         'scheduler_state_dict'     : scheduler.state_dict(),
         metric[0]                  : metric[1],
         'epoch'                    : epoch},
         path
    )

def load_model(path, model, metric= 'valid_acc', optimizer= None, scheduler= None):

    checkpoint = torch.load(path)
    model.load_state_dict(checkpoint['model_state_dict'])

    if optimizer != None:
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    if scheduler != None:
        scheduler.load_state_dict(checkpoint['scheduler_state_dict'])

    epoch   = checkpoint['epoch']
    metric  = checkpoint[metric]

    return [model, optimizer, scheduler, epoch, metric]

In [ ]:
# This is for checkpointing, if you're doing it over multiple sessions

last_epoch_completed = 0
start = last_epoch_completed
end = config["epochs"]
best_lev_dist = float("inf") # if you're restarting from some checkpoint, use what you saw there.
# epoch_model_path = #TODO set the model path( Optional, you can just store best one. Make sure to make the changes below )
# best_model_path = #TODO set best model path

# Always keep the path for the best model to ensure you can retrieve the best performing model
best_model_path ="/content/best_model_hw3.pth"


In [ ]:
torch.cuda.empty_cache()
gc.collect()

#TODO: Please complete the training loop

for epoch in range(0, config['epochs']):

    print("\nEpoch: {}/{}".format(epoch+1, config['epochs']))

    curr_lr = float(optimizer.param_groups[0]['lr']) #TODO

    train_loss              = train_model(model, train_loader, criterion, optimizer) #TODO
    valid_loss, valid_dist  = validate_model(model, val_loader, decoder, LABELS)#TODO
    scheduler.step(valid_dist)

    print("\tTrain Loss {:.04f}\t Learning Rate {:.07f}".format(train_loss, curr_lr))
    print("\tVal Dist {:.04f}%\t Val Loss {:.04f}".format(valid_dist, valid_loss))


    wandb.log({
        'train_loss': train_loss,
        'valid_dist': valid_dist,
        'valid_loss': valid_loss,
        'lr'        : curr_lr
    })

    # save_model(model, optimizer, scheduler, ['valid_dist', valid_dist], epoch, epoch_model_path)
    # wandb.save(epoch_model_path)
    # print("Saved epoch model")

    if valid_dist <= best_lev_dist:
        best_lev_dist = valid_dist
        save_model(model, optimizer, scheduler, ['valid_dist', valid_dist], epoch, best_model_path)
        wandb.save(best_model_path)
        print("Saved best model")
      # You may find it interesting to exlplore Wandb Artifcats to version your models
run.finish()


Epoch: 1/40


wandb: WARNING Saving files without folders. If you want to preserve subdirectories pass base_path to wandb.save, i.e. wandb.save("/mnt/folder/file.h5", base_path="/mnt")


	Train Loss 4.3655	 Learning Rate 0.0020000
	Val Dist 66.4590%	 Val Loss 3.6420
Saved best model

Epoch: 2/40


	Train Loss 3.7067	 Learning Rate 0.0020000
	Val Dist 66.0321%	 Val Loss 3.4735
Saved best model

Epoch: 3/40


	Train Loss 3.4236	 Learning Rate 0.0020000
	Val Dist 63.2707%	 Val Loss 3.2493
Saved best model

Epoch: 4/40


...

	Train Loss 0.5780	 Learning Rate 0.0000006
	Val Dist 8.0585%	 Val Loss 0.4737

Epoch: 38/40


	Train Loss 0.5792	 Learning Rate 0.0000001
	Val Dist 8.0550%	 Val Loss 0.4746

Epoch: 39/40


	Train Loss 0.5792	 Learning Rate 0.0000001
	Val Dist 8.0621%	 Val Loss 0.4732

Epoch: 40/40


	Train Loss 0.5790	 Learning Rate 0.0000001
	Val Dist 8.0400%	 Val Loss 0.4737
Saved best model


lr,███████▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_loss,█▇▆▆▄▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
valid_dist,███▇▃▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
valid_loss,██▇▆▃▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
lr,0.0
train_loss,0.579
valid_dist,8.04001
valid_loss,0.47365


# Generate Predictions and Submit to Kaggle

In [ ]:
#TODO: Make predictions

# Follow the steps below:
# 1. Create a new object for CTCBeamDecoder with larger (why?) number of beams
# 2. Get prediction string by decoding the results of the beam decoder

TEST_BEAM_WIDTH = 10 #TODO

test_decoder    = CTCBeamDecoder(
    labels=LABELS,
    model_path=None,
    alpha=0,
    beta=0,
    cutoff_top_n=40,
    cutoff_prob=1.0,
    beam_width=TEST_BEAM_WIDTH,
    num_processes=4,
    blank_id=0,
    log_probs_input=True
)#TODO
results = []

PATH_CHECKPOINT = "/content/best_model_hw3.pth"
checkpoint = torch.load(PATH_CHECKPOINT)
model.load_state_dict(checkpoint['model_state_dict'])

model.eval()
print("Testing")
for data in tqdm(test_loader):

    x, lx   = data
    print(x.shape)
    print(lx.shape)
    x       = x.to(device)

    with torch.no_grad():
        h, lh = model(x, lx)

    output = decode_prediction(h,lh,test_decoder,LABELS)
    results.extend(output)

    del x, lx, h, lh
    torch.cuda.empty_cache()

In [ ]:
data_dir = "/content/11-785-s24-hw3p2" + "/test-clean/random_submission.csv"
df = pd.read_csv(data_dir)
df.label = results
df.to_csv('submission.csv', index = False)

!kaggle competitions submit -c hw3p2asr-s24slack -f submission.csv -m "Message"


100% 206k/206k [00:00<00:00, 488kB/s]
Successfully submitted to HW3P2_ASR-S24(slack)